# Image Preprocessing

Importing Image Data Generator Library

In [42]:
pwd

'/home/wsuser/work'

In [3]:
!pip install keras

     |████████████████████████████████| 1.3 MB 30.1 MB/s eta 0:00:01


In [5]:
!pip install numpy

In [6]:
!pip install tensorflow

     |████████████████████████████████| 65 kB 9.0 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 46.7 MB/s eta 0:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.35.0
    Uninstalling grpcio-1.35.0:
      Successfully uninstalled grpcio-1.35.0
  Attempting uninstall: opt-einsum
    Found existing installation: opt-einsum 3.1.0
    Uninstalling opt-einsum-3.1.0:
      Successfully uninstalled opt-einsum-3.1.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 20210226132247
    Uninstalling flatbuffers-20210226132247:
      Successfully uninstalled flatbuffers-20210226132247


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Configure Image datagenerator class

In [8]:
#setting parameter for image data agumentationto the training data
train_datagen=ImageDataGenerator(rescale=1./225,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#image data agumenatation to test dataset
test_datagen=ImageDataGenerator(rescale=1./225)

Applying ImageDataGenerator Functionality to train and test Dataset

In [9]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_5778660795c64f0c9b7a116d67a50f9e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='8TfdObEBn9vHQOCTw4jgcxMQ2n7M7g4AAWNInWXyAyCm',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_5778660795c64f0c9b7a116d67a50f9e.get_object(Bucket='ecgimagebasedheartbeatclassificat-donotdelete-pr-7en7apd4ffji0o', Key='data.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [10]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [11]:
pwd

'/home/wsuser/work'

In [12]:
import os
filenames = os.listdir('/home/wsuser/work/data/train')

In [13]:
#performing data agumentation to train the dataset
x_train=train_datagen.flow_from_directory("/home/wsuser/work/data/train",target_size=(64,64),batch_size=32,class_mode='categorical')
#performing agumentation to test the dataset
x_test=test_datagen.flow_from_directory("/home/wsuser/work/data/test",target_size=(64,64),batch_size=32,class_mode='categorical')


Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


# Model Building

Importing Libraries

In [14]:

import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D

In [15]:
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

In [16]:
import tensorflow.keras

In [17]:
model= Sequential()

Adding CNN Layers

In [18]:
#adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

Adding Dense Layers

In [19]:
model.add(Dense(32))
model.add(Dense(6,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                200736    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

Configuring the learning process

In [20]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Train the Model

In [21]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
480/480 [==============================] - 71s 146ms/step - loss: 1.3116 - accuracy: 0.5481 - val_loss: 0.5941 - val_accuracy: 0.7893
Epoch 2/10
480/480 [==============================] - 67s 139ms/step - loss: 0.3517 - accuracy: 0.8970 - val_loss: 0.4653 - val_accuracy: 0.8708
Epoch 3/10
480/480 [==============================] - 67s 141ms/step - loss: 0.2686 - accuracy: 0.9221 - val_loss: 0.3997 - val_accuracy: 0.8781
Epoch 4/10
480/480 [==============================] - 67s 140ms/step - loss: 0.2425 - accuracy: 0.9266 - val_loss: 0.3374 - val_accuracy: 0.9033
Epoch 5/10
480/480 [==============================] - 66s 138ms/step - loss: 0.2194 - accuracy: 0.9349 - val_loss: 0.3256 - val_accuracy: 0.9093
Epoch 6/10
480/480 [==============================] - 69s 143ms/step - loss: 0.1899 - accuracy: 0.9444 - val_loss: 0.3106 - val_accuracy: 0.9021
Epoch 7/10
480/480 [==============================] - 69s 143ms/step - loss: 0.1904 - accuracy: 0.9421 - val_loss: 0.3181 - val_ac

Saving the model

In [23]:
model.save("ECG.h5")

In [24]:
!tar -zcvf image-classification-model_new.tgz ECG.h5

ECG.h5


In [25]:
ls -1


data/
ECG.h5
image-classification-model_new.tgz


In [26]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 28.4 MB/s eta 0:00:01


In [27]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "xySVc8viB7jco7hmwm2LthKwWv_kOicPJC7AZtKWL8uu"
}

client = APIClient(wml_credentials)

In [28]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [29]:
space_uid = guid_from_space_name(client, space_name='ECG')
print("Space UID = " + space_uid)

Space UID = ef13ab5e-6699-4647-a31e-f0ced40eefbd


In [30]:
client.set.default_space(space_uid)

'SUCCESS'

In [31]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [32]:
software_spec_uid= client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7-horovod")
software_spec_uid

'1092590a-307d-563d-9b62-4eb7d64b3f22'

In [33]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [43]:
model_id

'55a6cf80-620d-41ee-a4ad-65c12e8a8d97'

Test the Model